In [ ]:
# pip install apiclient

In [ ]:
# pip install oauth2client

In [ ]:
# pip install google-cloud-bigquery

In [1]:
import pandas as pd
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
from google.cloud import bigquery 
from google.oauth2 import service_account

In [29]:
credentialsPath = r'/Users/rajathadri_as/Documents/GoogleAPI/fmg-datastores-a734912841ea.json'

In [30]:
MEASUREMENT_ID = 'ga:345528495'

In [31]:
# credentials = service_account.Credentials.from_service_account_file(credentialsPath)

In [32]:
credentials = service_account.Credentials.from_service_account_file(
    credentialsPath,
    scopes=['https://www.googleapis.com/auth/analytics.readonly']
)

In [33]:
# Create the Analytics Reporting API V4 service object
analytics = build('analyticsreporting', 'v4', credentials=credentials)

In [34]:
date_range = '7daysAgo'  # Data range for the report
metrics = [{'expression': 'ga:sessions'}, {'expression': 'ga:pageviews'}]  # Metrics to retrieve
dimensions = [{'name': 'ga:date'}]  # Dimensions to retrieve

In [35]:
request = {
    'reportRequests': [
        {
            'viewId': MEASUREMENT_ID,
            'dateRanges': [{'startDate': date_range, 'endDate': 'today'}],
            'metrics': metrics,
            'dimensions': dimensions
        }
    ]
}

In [36]:
# calling API
response = analytics.reports().batchGet(body=request).execute()

HttpError: <HttpError 403 when requesting https://analyticsreporting.googleapis.com/v4/reports:batchGet?alt=json returned "User does not have sufficient permissions for this profile.". Details: "User does not have sufficient permissions for this profile.">

In [16]:
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
from google.auth.exceptions import RefreshError

# Path to the service account key file
SERVICE_ACCOUNT_FILE = '/Users/rajathadri_as/Documents/GoogleAPI/fmg-datastores-a734912841ea.json'

# Define the Google Analytics View ID
VIEW_ID = 'ga:345528495'  # Replace with your Google Analytics View ID

# Create a credentials object from the service account key file
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE,
    scopes=['https://www.googleapis.com/auth/analytics.readonly']
)

try:
    # Create the Analytics Reporting API V4 service object
    analytics = build('analyticsreporting', 'v4', credentials=credentials)

    # Query parameters
    date_range = '7daysAgo'  # Data range for the report
    metrics = [{'expression': 'ga:sessions'}, {'expression': 'ga:pageviews'}]  # Metrics to retrieve
    dimensions = [{'name': 'ga:date'}]  # Dimensions to retrieve

    # Request data from the API
    request = {
        'reportRequests': [
            {
                'viewId': VIEW_ID,
                'dateRanges': [{'startDate': date_range, 'endDate': 'today'}],
                'metrics': metrics,
                'dimensions': dimensions
            }
        ]
    }

    # Call the API
    response = analytics.reports().batchGet(body=request).execute()

    # Parse the API response
    def parse_response(response):
        rows = []
        for report in response.get('reports', []):
            columnHeader = report.get('columnHeader', {})
            dimensionHeaders = columnHeader.get('dimensions', [])
            metricHeaders = [entry['name'] for entry in columnHeader['metricHeader']['metricHeaderEntries']]
            for row in report.get('data', {}).get('rows', []):
                dimensions = row.get('dimensions', [])
                dateRangeValues = row.get('metrics', [])
                row_data = {}
                for header, dimension in zip(dimensionHeaders, dimensions):
                    row_data[header] = dimension
                for i, values in enumerate(dateRangeValues):
                    for metric, value in zip(metricHeaders, values['values']):
                        row_data[metric] = value
                rows.append(row_data)
        return rows

    # Convert the parsed response to a DataFrame
    df = pd.DataFrame(parse_response(response))

    # Display the DataFrame
    print(df.head())

except RefreshError as e:
    print("Error: Please check your credentials or the service account permissions.")
except Exception as e:
    print("Error:", e)


Error: <HttpError 403 when requesting https://analyticsreporting.googleapis.com/v4/reports:batchGet?alt=json returned "User does not have sufficient permissions for this profile.". Details: "User does not have sufficient permissions for this profile.">


In [22]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = '/Users/rajathadri_as/Documents/GoogleAPI/fmg-datastores-a734912841ea.json' #The document receive while creating the API
VIEW_ID = 'ga:345528495' #the view from your Google Analytics

In [23]:
#Initializing report
def initialize_analyticsreporting():
  credentials = ServiceAccountCredentials.from_json_keyfile_name(
      KEY_FILE_LOCATION, SCOPES)
  analytics = build('analyticsreporting', 'v4', credentials=credentials)
  return analytics

In [24]:
#Get one report page
def get_report(analytics, pageTokenVar):
  return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': MEASUREMENT_ID,
          'dateRanges': [{'startDate': '600daysAgo', 'endDate': 'yesterday'}],
          'metrics': [{'expression': 'ga:sessions'}, {'expression': 'ga:Pageviews'}], #, {'expression': 'ga:Goal completions'} didn't work.
          'dimensions': [{'name': 'ga:Date'}],
          'pageSize': 10000,
          'pageToken': pageTokenVar,
          'samplingLevel': 'LARGE'
        }]
      }
  ).execute()

In [25]:
def handle_report(analytics,pagetoken,rows):  
    response = get_report(analytics, pagetoken)

    #Header, Dimensions Headers, Metric Headers 
    columnHeader = response.get("reports")[0].get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

    #Pagination
    pagetoken = response.get("reports")[0].get('nextPageToken', None)
    
    #Rows
    rowsNew = response.get("reports")[0].get('data', {}).get('rows', [])
    rows = rows + rowsNew
    print("len(rows): " + str(len(rows)))

    #Recursivly query next page
    if pagetoken != None:
        return handle_report(analytics,pagetoken,rows)
    else:
        #nicer results
        nicerows=[]
        for row in rows:
            dic={}
            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])

            for header, dimension in zip(dimensionHeaders, dimensions):
                dic[header] = dimension

            for i, values in enumerate(dateRangeValues):
                for metric, value in zip(metricHeaders, values.get('values')):
                    if ',' in value or ',' in value:
                        dic[metric.get('name')] = float(value)
                    else:
                        dic[metric.get('name')] = int(value)
            nicerows.append(dic)
        return nicerows

In [26]:
#Start
def main():    
    analytics = initialize_analyticsreporting()
     
    global dfanalytics
    dfanalytics = []
 
    rows = []
    rows = handle_report(analytics,'0',rows)
 
    dfanalytics = pd.DataFrame(list(rows))

In [27]:
main()

HttpError: <HttpError 403 when requesting https://analyticsreporting.googleapis.com/v4/reports:batchGet?alt=json returned "User does not have sufficient permissions for this profile.". Details: "User does not have sufficient permissions for this profile.">

In [28]:
print(dfanalytics)

[]
